In [2]:
## -- Trova il primo swing

def find_first_swing(signal):
    found = False
    swing_onset = 0
    for i in range(0,signal.shape[0]): 
        if not found:
            swing_onset += 1
        if signal[i,-1] == 1:
            found = True
    return swing_onset

def find_first_swing_baso(signal):
    found = False
    swing_onset = 0
    for i in range(0,signal.shape[0]): 
        if not found:
            swing_onset += 1
        if signal[i] == 1:
            found = True
    return swing_onset

In [111]:
## -- Elimina i falsi stance in mezzo agli swing - i falsi 0 in mezzo agli 1

def delete_false_stance(signal, thr, only_baso):
    cont = 0
    stop = False
    
    if only_baso:
        ## -- Trovo tutte le transizioni 1 -> 0
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i] == 1 and signal[i+1] == 0:
                transitions.append(i) 
        
        ## -- Controllo quali siano false e le salvo
        false_transitions = []
        for k in transitions:
            if any(signal[k : k + thr]) == 1:
                stop = False
                for j in range(k + 1 , k + thr):
                    if not stop:
                        cont += 1
                    if signal[j] == 1:
                        stop = True     
            if stop:
                false_transitions.append(k + 1)    # Mi salvo le transizioni finte
        
        
        ## -- Agisco sulle false transizioni 0->1 trasformandole in 1 fino a dove prima era 0
        cont = 0
        for tran in false_transitions:
            print(tran)
            stop = False
            for t in range(tran + 1 , tran + thr):
                if not stop:
                    cont += 1
                if signal[t] == 1:
                    stop = True
                    signal[tran : tran + cont] = 1
                    cont = 0
                
    if not only_baso:
    ## -- Trovo tutte le transizioni 1 -> 0
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i,-1] == 1 and signal[i+1,-1] == 0:
                transitions.append(i) 
        
        ## -- Controllo quali siano false e le salvo
        false_transitions = []
        for k in transitions:
            if any(signal[k : k + thr,-1]) == 1:
                stop = False
                for j in range(k + 1 , k + thr):
                    if not stop:
                        cont += 1
                    if signal[j,-1] == 1:
                        stop = True     
            if stop:
                false_transitions.append(k + 1)    # Mi salvo le transizioni finte
        
        
        ## -- Agisco sulle false transizioni 0->1 trasformandole in 1 fino a dove prima era 0
        cont = 0
        for tran in false_transitions:
            print(tran)
            stop = False
            for t in range(tran + 1 , tran + thr):
                if not stop:
                    cont += 1
                if signal[t,-1] == 1:
                    stop = True
                    signal[tran : tran + cont,-1] = 1
                    cont = 0
    return signal

In [112]:
## -- Elimina i falsi swing (non fisiologici) in base alla threshold che gli si da

# Devo trovare le altre transizioni - stance to swing - 0 -> 1
def delete_false_swing(signal, thr, only_baso):
    cont = 0
    stop = False
    
    if only_baso:
       ## -- Trovo tutte le transizioni 1 -> 0
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i] == 0 and signal[i+1] == 1:
                transitions.append(i) 
        
        ## -- Controllo quali siano false e le salvo
        false_transitions = []
        for k in transitions:
            if any(signal[k : k + thr]) == 0:
                stop = False
                for j in range(k + 1 , k + thr):
                    if not stop:
                        cont += 1
                    if signal[j] == 1:
                        stop = True     
            if stop:
                false_transitions.append(k + 1)    # Mi salvo le transizioni finte
        
        
        ## -- Agisco sulle false transizioni 0->1 trasformandole in 1 fino a dove prima era 0
        cont = 0
        for tran in false_transitions:
            print(tran)
            stop = False
            for t in range(tran + 1 , tran + thr):
                if not stop:
                    cont += 1
                if signal[t] == 0:
                    stop = True
                    signal[tran : tran + cont] = 0
                    cont = 0        
        
    if not(only_baso):
        ## -- Trovo tutte le transizioni 1 -> 0
        transitions = []
        for i in range(0,signal.shape[0] - 1):
            if signal[i,-1] == 0 and signal[i+1,-1] == 1:
                transitions.append(i) 
        
        ## -- Controllo quali siano false e le salvo
        false_transitions = []
        for k in transitions:
            if any(signal[k : k + thr,-1]) == 0:
                stop = False
                for j in range(k + 1 , k + thr):
                    if not stop:
                        cont += 1
                    if signal[j,-1] == 1:
                        stop = True     
            if stop:
                false_transitions.append(k + 1)    # Mi salvo le transizioni finte
        
        
        ## -- Agisco sulle false transizioni 0->1 trasformandole in 1 fino a dove prima era 0
        cont = 0
        for tran in false_transitions:
            stop = False
            for t in range(tran + 1 , tran + thr):
                if not stop:
                    cont += 1
                if signal[t,-1] == 0:
                    stop = True
                    signal[tran : tran + cont,-1] = 0
                    cont = 0        
    return signal

In [114]:
## -- Clean Basograhic Signal
# Questo script mi serve per eliminare tutti i "passi falsi" :-)

import numpy as np
from numpy import genfromtxt as csv_load
# import matplotlib.pyplot as plt
import pandas as pd

thr = 200

tipo = np.dtype(np.float32)    # Per farglielo caricare 

in_directory = '../subjects/'
prefix = 'Biphase_Subject'
suffix = '_norm.csv'
out_directory = '../subjects/clean_baso/'
out_path = out_directory + 'Real/'
out_prefix = 's'
baso_suffix = '_baso_real.csv'
subjects = 26

for subject in [10,13,14,17]:
#for subject in range(1, subjects + 1):
    # Carica il segnale (è un NUMPY)
    signal = csv_load(in_directory + prefix + str(subject) + suffix, dtype = tipo, delimiter = ',')

    # Calcola il primo swing e aggiorna il vettore
    lower_bound = find_first_swing(signal)
    signal = signal[lower_bound:,:]
    
    # Cancella le false transizioni 
    signal = delete_false_stance(signal, thr,only_baso = False)
    signal = delete_false_swing(signal, thr, only_baso = False)
    baso = signal[:,-1]
    
    signal = pd.DataFrame(signal)
    baso = pd.DataFrame(baso)
    signal = signal.to_csv(out_directory + out_prefix + str(subject) + suffix, index = None, header = None)    # Salvo il file completo
    baso = baso.to_csv(out_path + out_prefix + str(subject) + baso_suffix, index = None, header = None)    # Salvo solo il basografico

340718
308511
344175
132938
305709
341373
448745
63285
175291
340648
448020


In [119]:
## -- Per i segnali predetti

import numpy as np
from numpy import genfromtxt as csv_load
# import matplotlib.pyplot as plt
import pandas as pd

tipo = np.dtype(np.float32)    # Per farglielo caricare 
thr = 200

in_path = '../subjects/min-max/Predicted_Baso/FF2/'
subject = 17
file = 's' + str(subject) + '_predicted.csv'
out_directory = '../subjects/clean_baso/Predicted/'
out_prefix = 's'
suffix = '_predicted_clean.csv'

signal = csv_load(in_path + file, dtype = tipo, delimiter = ',')
#lower_bound = find_first_swing_baso(signal)
# signal = signal[lower_bound:]

signal = delete_false_stance(signal, thr, only_baso = True)
signal = delete_false_swing(signal, thr, only_baso = True)
signal

signal = pd.DataFrame(signal)
signal = signal.to_csv(out_directory + out_prefix + str(subject) + suffix, index = None, header = None)

OSError: ../subjects/min-max/Predicted_Baso/FF2/s17_predicted.csv not found.

In [38]:
a = [1, 2, 3, 4, 5]
a.remove(a[2])
a

[1, 2, 4, 5]